In [100]:
import sys
sys.path.append('/home/sakshmeno/Documents/GitHub/C-RNN-approach')

from Utils import dataframe_gen
import numpy as np


In [101]:
# df = dataframe_gen.dataframe_init()
df_obj = dataframe_gen
df = df_obj.dataframe_init()

In [102]:
df.head()

,Lines,Label
0,<start> <end>,Secure
1,<start> <end>,Secure
2,<start> #include <Compiler.h> <end>,Secure
3,<start> #include <TimeDelay.h> <end>,Secure
4,"<start> #include ""HardwareProfile.h"" <end>",Secure


In [103]:
x_vec = df['Lines']
y_vec = df['Label']

In [104]:

vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token

for line in np.asarray(x_vec):
  tokens = list(line.split())
  for token in tokens:
    if token not in vocab:
      vocab[token] = index
      index += 1
# vocab_size = len(vocab)
# inverse_vocab = {index: token for token, index in vocab.items()}
# print(inverse_vocab)
len(vocab.keys())

# for i in (np.asarray(x_vec)):
    

3746

In [105]:
vocab

{'<pad>': 0,
 '<start>': 1,
 '<end>': 2,
 '#include': 3,
 '<Compiler.h>': 4,
 '<TimeDelay.h>': 5,
 '"HardwareProfile.h"': 6,
 '"Peripherals/Nand.h"': 7,
 '"Peripherals/Accel.h"': 8,
 '#ifdef': 9,
 'USE_GYRO': 10,
 '"Peripherals/Gyro.h"': 11,
 '#endif': 12,
 '"Peripherals/Rtc.h"': 13,
 '"USB/USB.h"': 14,
 '"USB/usb_function_msd.h"': 15,
 '"USB/usb_function_cdc.h"': 16,
 '"MDD': 17,
 'File': 18,
 'System/FSIO.h"': 19,
 '"Usb/USB_CDC_MSD.h"': 20,
 '"Ftl/FsFtl.h"': 21,
 '"Utils/FsUtils.h"': 22,
 '"Utils/Fifo.h"': 23,
 '"Utils/Util.h"': 24,
 '"Peripherals/Analog.h"': 25,
 '"Settings.h"': 26,
 '"Logger.h"': 27,
 '"AX3': 28,
 'SelfTest.h"': 29,
 'void': 30,
 'RunAttached(void)<end>': 31,
 'RunLogging(void)<end>': 32,
 'TimedTasks(void)<end>': 33,
 'LedTasks(void)<end>': 34,
 'RunTestSequence(void)<end>': 35,
 '__attribute__((interrupt,auto_psv))': 36,
 '_DefaultInterrupt(void)': 37,
 '{': 38,
 'static': 39,
 'unsigned': 40,
 'int': 41,
 'INTCON1val<end>': 42,
 'LED_SET(LED_MAGENTA)<end>': 43,

In [106]:
def one_hot_encode(categories, category_to_index):
    gloabl_OHV_dictionary = {}
    num_categories = len(categories)
    num_indices = len(category_to_index)
    one_hot_vectors = np.zeros((num_categories, num_indices))
    for i, category in enumerate(categories):
        index = category_to_index.get(category)
        if index is not None:
            one_hot_vectors[i, index] = 1
            gloabl_OHV_dictionary[category] = one_hot_vectors[i]
    return one_hot_vectors, gloabl_OHV_dictionary

global_one_hot_vectors, gloabl_OHV_dictionary = one_hot_encode(vocab.keys(), vocab)

In [112]:
df[df['Label']=='Insecure']

,Lines,Label
170,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
233,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
250,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
322,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
327,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
...,...,...
4946,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
5026,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
5080,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure
5092,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",Insecure


In [107]:
for line in range(df.shape[0]):
    df.loc[line].Lines = df.loc[line].Lines.split()
    for element_index in range(len(df.loc[line].Lines)):
        df.loc[line].Lines[element_index] = gloabl_OHV_dictionary[df.loc[line].Lines[element_index]]

max_len = 0
for i in range(df.shape[0]):
    if  len(df.loc[i].Lines) > max_len:
        max_len = (len(df.loc[i].Lines))
for i in enumerate(df.loc[:]):
    for iter in range(0, max_len - len(i[1])):
        df.loc[i[0]].Lines.insert(-1, gloabl_OHV_dictionary['<pad>'])